# Programa para plotar a comparacao do ciclo diurno de temperatura e umidade relativa modelo x observacao

In [1]:
%matplotlib inline
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as md
import seaborn as sns
import datetime
import glob
import os

In [2]:
obs_path = '...'

In [3]:
obsvs = list(glob.glob(os.path.join(obs_path, 'Observacao*.txt'))) # Identifica todos os dados de observação com formato Observacao_estacao(nome).txt
stations = ["Guarulhos","Campo de Marte","Mirante de Santana","IAG","Congonhas","Barueri","Capao Redondo","Interlagos","Ponte dos Remedios", 
            "Parque D Pedro II","Pinheiros"]

In [1]:
def make_date (ano, mes, dia, hora):
    """
    Função para combinar ano, mes, dia e hora e tranformar em data
    """
    somedates = ano*10000000000 + mes*100000000 + dia*1000000 + hora*10000
    dates = [datetime.datetime.strptime(str(int(date)),'%Y%m%d%H%M%S') for date in somedates]
    return dates

In [25]:
def plot_data(obs,model,var,**kwargs):
    fig, ax = plt.subplots(figsize=(6, 6),facecolor="w")
    sns.set_style("dark")
    sns.lineplot(
        data=obs,x=obs['DATA'],y=obs[var],color='k',linewidth=2.0, 
         ax=ax, label='Observação'
        )
    sns.lineplot(
        data=model,x=model['DATA'],y=model[var],color='r',linewidth=2.0, linestyle='--',
         ax=ax, label='Modelo'
        )
    ax.set_xbound(lower=datetime.datetime(2016,12,18,0,0,0), upper=datetime.datetime(2016,12,18,23,0,0))
    ax.xaxis.set_major_locator(md.HourLocator(byhour=range(0,24,3)))
    ax.xaxis.set_major_formatter(md.DateFormatter('%H'))
    ax.yaxis.grid(True)
    ax.xaxis.grid(True)
    ax.tick_params(axis='both', which='major', labelsize=12)
    sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
    plt.setp(ax.get_legend().get_texts(), fontsize='15')
    ax.set_xlabel(r'Horário (UTC)',size=15)
    ax.set_ylabel(kwargs.get('label'),  size = 15)
    plt.title(kwargs.get('estacao'), size = 15)
    plt.savefig('CicloDiurno_' + kwargs.get('estacao') + '_' + var, dpi=300, bbox_inches='tight')
    plt.close()

In [16]:
dfs = []
for obsv in obsvs:
    dados_obs = pd.read_table(obsv) # Ler dados de observacao
    ano_obs = dados_obs['ANO']
    mes_obs = dados_obs['MES']
    dia_obs = dados_obs['DIA']
    hora_obs = dados_obs['HORA']
    dates = make_date(ano_obs, mes_obs, dia_obs, hora_obs)
    dfs.append(pd.concat(axis=0, ignore_index=True, objs=[
                         pd.DataFrame.from_dict({'ESTACAO': dados_obs['ESTACAO'],
                                                 'DATA': dates,
                                                 'TEMP': dados_obs['TEMP(C)'],
                                                 'UR': dados_obs['UR(%)']
                                                 })
                            ]))
df = pd.concat(dfs, ignore_index=True)
df

,ESTACAO,DATA,TEMP,UR
0,A701,2016-12-18 00:00:00,20.2,75.0
1,A701,2016-12-18 01:00:00,20.0,76.0
2,A701,2016-12-18 02:00:00,19.8,77.0
3,A701,2016-12-18 03:00:00,19.7,77.0
4,A701,2016-12-18 04:00:00,19.2,79.0
...,...,...,...,...
314,SBSP,2016-12-18 20:00:00,24.0,69.1
315,SBSP,2016-12-18 21:00:00,23.0,69.0
316,SBSP,2016-12-18 22:00:00,23.0,69.0
317,SBSP,2016-12-18 23:00:00,22.0,73.3


In [17]:
df.replace({'ESTACAO':{'A701':"Mirante de Santana","SBGR": "Guarulhos","SBSP":"Congonhas","SBMT":"Campo de Marte",
                      "A755": "Barueri"}}, inplace = True) #substitui o codigo das estacoes pelos nomes
df

,ESTACAO,DATA,TEMP,UR
0,Mirante de Santana,2016-12-18 00:00:00,20.2,75.0
1,Mirante de Santana,2016-12-18 01:00:00,20.0,76.0
2,Mirante de Santana,2016-12-18 02:00:00,19.8,77.0
3,Mirante de Santana,2016-12-18 03:00:00,19.7,77.0
4,Mirante de Santana,2016-12-18 04:00:00,19.2,79.0
...,...,...,...,...
314,Congonhas,2016-12-18 20:00:00,24.0,69.1
315,Congonhas,2016-12-18 21:00:00,23.0,69.0
316,Congonhas,2016-12-18 22:00:00,23.0,69.0
317,Congonhas,2016-12-18 23:00:00,22.0,73.3


In [19]:
latlon = np.array([[-23.44,-46.47],[-23.50,-46.63], #Guarulhos, Campo de Marte
                   [-23.49, -46.62], [-23.65,-46.62], #Mirante de Santana, IAG
            [-23.62,-46.65],[-23.5389,-46.86945],[-23.6655,-46.78207], #Congonhas, Barueri (INMET), Capao Redondo (CETESB)
            [-23.68097,-46.67577],[-23.51864,-46.74302], # Interlagos, Pontes dos remedios (CETESB)
            [-23.54602,-46.62717],[-23.56159,-46.70206]]) #Parque D.Pedro II, Pinheiros (CETESB) 


In [20]:
dataFile = '...'


In [21]:
dfs = []
for i in range(len(latlon)):
    #Leitura dos dados do modelo a cada 1 hora
    ds = xr.open_dataset(dataFile).sel(lat=latlon[i,0], lon=latlon[i,1], method="nearest").isel(time=slice(288,577,12))
    dfs.append(pd.concat(axis=0, ignore_index=True, objs=[
                         pd.DataFrame.from_dict({'ESTACAO': stations[i],
                                                 'DATA': ds['time'],
                                                 'TEMP': ds['tempc2m'],
                                                 'UR': ds['rh'].isel(lev=0)
                                                 })
                            ]))
df_model = pd.concat(dfs, ignore_index=True)

In [22]:
df_model

,ESTACAO,DATA,TEMP,UR
0,Guarulhos,2016-12-18 00:00:00,18.998474,80.603447
1,Guarulhos,2016-12-18 01:00:00,18.350403,84.725494
2,Guarulhos,2016-12-18 02:00:00,17.999634,85.732948
3,Guarulhos,2016-12-18 03:00:00,17.779968,86.403862
4,Guarulhos,2016-12-18 04:00:00,17.611115,87.358620
...,...,...,...,...
270,Pinheiros,2016-12-18 20:00:00,20.379578,76.148590
271,Pinheiros,2016-12-18 21:00:00,21.412262,75.123772
272,Pinheiros,2016-12-18 22:00:00,21.171295,80.736176
273,Pinheiros,2016-12-18 23:00:00,20.866974,84.069496


In [26]:
for station in stations:
    plot_data(df.loc[df['ESTACAO']==station],df_model.loc[df_model['ESTACAO']==station],'TEMP',label='Temperatura ($^\circ$C)',
            estacao=station)
    plot_data(df.loc[df['ESTACAO']==station],df_model.loc[df_model['ESTACAO']==station],'UR',label='Umidade Relativa (%)',
            estacao=station)